#### [ 모델 저장 및 로딩 ]
- 2가지 형태 저장
    - 전체 저장
    - 모델의 파라미터만 저장

- 2가지 형태 로딩
    - 전체 저장 모델 파일 ==> 로딩으로 사용 가능
    - 모델 파라미터만 저장 ==> 모델 객체 생성 후 층별 파라미터 삽입


[1] 모듈 로딩 및 데이터 준비<hr>

In [63]:
## [1-1] 모듈 로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import utils as uf


In [64]:
## [1-2] 데이터 준비
featureTS = torch.tensor([[1,5],[2,6], [3,7], [4,8],[5,9]], dtype=torch.float32)
targetTS = torch.tensor([[6], [8], [10], [12], [14]], dtype=torch.float32)

[2] 모델 클래스 정의 <hr>

In [65]:
class TEST(nn.Module):
    def __init__(self) :
        super().__init__()
        self.fc1 = nn.Linear(2, 16)
        self.fc2 = nn.Linear(16, 4)
        self.out = nn.Linear(4, 1)
        
    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = F.relu(self.fc2(out))
        return self.out(out)

In [66]:
## 설정값들
EPOCHS = 10
BS = 2
LR = 0.1

## 저장 모델 파일명
ALL_MODEL = './all_model.pt'    ## 모델 전체 확장자
WEIGHTS_MODEL = './weights'     ## 파라미터 저장 확장자 pth

DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"


In [67]:
## 인스턴스들
model = TEST().to(DEVICE)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr =LR)

dataDS  = TensorDataset(featureTS, targetTS)
dataDL  = DataLoader(dataDS, batch_size=BS, shuffle=True)

dataDS  = TensorDataset(featureTS*1.2, targetTS*1.1)
validDL = DataLoader(dataDS, batch_size=BS)

In [68]:
for x, y in dataDL :
    print(x, y, sep='\n')
    break
print()
for x, y in validDL :
    print(x, y, sep='\n')
    break

tensor([[3., 7.],
        [1., 5.]])
tensor([[10.],
        [ 6.]])

tensor([[1.2000, 6.0000],
        [2.4000, 7.2000]])
tensor([[6.6000],
        [8.8000]])


In [74]:
## 학습 진행 ==================
BEST_LOSS = 100.
DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
for e in range(EPOCHS) :
    ## 학습진행
    train_loss, train_acc = uf.train_one_epoch(model,
                                               dataDL,
                                               loss_fn,
                                               optimizer,
                                               DEVICE)
    
    ## 검증 진행
    valid_loss, valid_acc = uf.evaluate(model, validDL, loss_fn, DEVICE)
    
    #- 모델과 가중치 파일 저장
    if BEST_LOSS > valid_loss : 
        ## 모델 전체 저장
        torch.save(model, ALL_MODEL)
        ## 파라미터만 저장
        torch.save(model.state_dict(), f"./{WEIGHTS_MODEL}_{e}.pth")
        ## 기준 loss 업데이트
        BEST_LOSS = valid_loss
        
    # 학습상태 출력
    print(f'[loss] train : valid = {train_loss} : {valid_loss} [acc] train : valid = {train_acc} : {valid_acc}')
    

[loss] train : valid = 0.003634367970516905 : 0.9186471700668335 [acc] train : valid = 0.0 : 0.0
[loss] train : valid = 0.0011047897045500577 : 0.9016615629196167 [acc] train : valid = 0.0 : 0.0
[loss] train : valid = 0.0003346938727190718 : 0.866930079460144 [acc] train : valid = 0.0 : 0.0
[loss] train : valid = 0.00016592639149166645 : 0.9377578020095825 [acc] train : valid = 0.0 : 0.0
[loss] train : valid = 0.0002254185441415757 : 0.8447158217430115 [acc] train : valid = 0.0 : 0.0
[loss] train : valid = 0.0003814853815129027 : 0.9147611618041992 [acc] train : valid = 0.0 : 0.0
[loss] train : valid = 0.000211855996894883 : 0.9008216619491577 [acc] train : valid = 0.0 : 0.0
[loss] train : valid = 6.66193533106707e-05 : 0.8691281318664551 [acc] train : valid = 0.0 : 0.0
[loss] train : valid = 6.314620259217918e-05 : 0.9068525791168213 [acc] train : valid = 0.0 : 0.0
[loss] train : valid = 4.817016688321019e-05 : 0.8784166216850281 [acc] train : valid = 0.0 : 0.0


[4] 모델 파일 사용 <hr>

In [77]:
## [4-1] 가중치 저장 파일 로딩
params = torch.load("./weights_0.pth", weights_only=True)

tModel = TEST()
tModel.load_state_dict(params)

<All keys matched successfully>

In [81]:
## [4-2] 전체 모델 저장 파일 로딩
allModel = torch.load(ALL_MODEL, weights_only=False)
allModel

TEST(
  (fc1): Linear(in_features=2, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=4, bias=True)
  (out): Linear(in_features=4, out_features=1, bias=True)
)